In [1]:
# Import des librairies

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from peft import get_peft_model, LoraConfig
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from depth_anything_v2.dpt import DepthAnythingV2

xFormers not available
xFormers not available


In [2]:
# Vérification de la disponibilité du GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# 1. Chargement du modèle Depth Anything

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

encoder = "vits"

model = DepthAnythingV2(**model_configs[encoder ])
model.load_state_dict(torch.load(f'checkpoints/depth_anything_v2_{encoder}.pth', map_location='cpu'))
model = model.to(device).eval()

C:\Users\estelle\AppData\Local\Temp\ipykernel_20184\2106644882.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'checkpoints/depth_anyt

In [7]:
# Geler tous les paramètres pour LoRA

for param in model.parameters():
    param.requires_grad = False

In [9]:
model

DepthAnythingV2(
  (pretrained): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
    

In [8]:
# 2. Configuration LoRA

lora_config = LoraConfig(
    r=8,  # Rang faible
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Couches à modifier
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

ValueError: Target modules {'v_proj', 'q_proj'} not found in the base model. Please check the target modules and try again.

In [ ]:
# 3. Préparation des données (à adapter selon vos données)

class DepthDataset(Dataset):
    def __init__(self, images, depth_maps):
        self.images = images
        self.depth_maps = depth_maps

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        depth_map = self.depth_maps[idx]
        return image, depth_map


In [ ]:
# Exemple de données (remplacez par votre chargement réel)
train_data = DepthDataset(images=[...], depth_maps=[...])
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

In [ ]:
# 4. Fine-tuning
optimizer = torch.optim.AdamW(model.parameter

In [ ]:
model.train()
for epoch in range(10):
    epoch_loss = 0
    for images, depth_maps in tqdm(train_loader):
        images = images.to(device)
        depth_maps = depth_maps.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, depth_maps)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_loader):.4f}")

In [ ]:
# 5. Sauvegarde du modèle
model.save_pretrained("./fine_tuned_depth_anything")
tokenizer.save_pretrained("./fine_tuned_depth_anything")

In [ ]:
# 6. Évaluation des performances
model.eval()
with torch.no_grad():
    for images, depth_maps in DataLoader(train_data, batch_size=16):
        images = images.to(device)
        depth_maps = depth_maps.to(device)
        outputs = model(images)
        # Ajouter ici vos métriques d'évaluation
        print("Evaluation metrics here...")